In [61]:
from utils.sessao_spark import sessao_spark
import pyspark.pandas as ps

In [62]:
spark = sessao_spark("convert-json-to-parquet")

In [63]:
# minio_bucket = "production/landing"
# arquivo = "yelp/business/business"
# tamanho_arquivo_mb = 134 


# minio_bucket = "production/landing"
# arquivo = "yelp/checkin/checkin"
# tamanho_arquivo_mb = 274 

# minio_bucket = "production/landing"
# arquivo = "yelp/tip/tip"
# tamanho_arquivo_mb = 172

# minio_bucket = "production/landing"
# arquivo = "yelp/user/user"
# tamanho_arquivo_mb = 3000 

minio_bucket = "production/landing"
arquivo = "yelp/review/review"
tamanho_arquivo_mb = 5000 

In [64]:
print(spark.version)

3.5.0


In [65]:
df = spark.read.json(f"s3a://{minio_bucket}/{arquivo}.json")

In [66]:
df_pandas = df.to_pandas_on_spark()
df_pandas.head()

/usr/local/spark/python/pyspark/sql/dataframe.py:5719: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,XQfwVwDr-v0ZS3_CbbE5Xw,0,2018-07-07 22:09:11,0,KU_O5udG6zpxOg-VcAEodg,3.0,"If you decide to eat here, just be aware it is...",0,mh_-eMZ6K5RLWhZyISBhwA
1,7ATYjTIgM3jUlt4UM3IypQ,1,2012-01-03 15:28:18,0,BiTunyQ73aT9WBnpR9DZGw,5.0,I've taken a lot of spin classes over the year...,1,OyoGAe7OKpv6SyGZT5g77Q
2,YjUWPpI6HXG530lwP-fb2A,0,2014-02-05 20:30:30,0,saUsX_uimxRlCVr67Z4Jig,3.0,Family diner. Had the buffet. Eclectic assortm...,0,8g_iMtfSiwikVnbP2etR0A
3,kxX2SOes4o-D3ZQBkiMRfA,1,2015-01-04 00:01:03,0,AqPFMleE6RsU23_auESxiA,5.0,"Wow! Yummy, different, delicious. Our favo...",1,_7bHUi9Uuf5__HHc_Q8guQ
4,e4Vwtrqf-wpJfwesgvdgxQ,1,2017-01-14 20:54:15,0,Sx8TMOWLNuJBWer-0pcmoA,4.0,Cute interior and owner (?) gave us tour of up...,1,bcjbaE6dDog4jkNY91ncLQ


In [67]:
particoes = int(round(tamanho_arquivo_mb / 128, 0))
df = df.repartition(particoes)

In [68]:
# Se você já tem um DataFrame 'df' carrega
print(f"s3a://{minio_bucket}/parquet/{arquivo}")
(df.write
   .mode("overwrite")   
   .parquet(f"s3a://{minio_bucket}/parquet/{arquivo}")
)

s3a://production/landing/parquet/yelp/review/review


In [69]:
spark.stop()